In [11]:
from roler.model import ModelPrior, ModelParams
from roler.distributions import *

In [ ]:
prior = ModelPrior(
    individuals_local=IntDistribution(100, 10**8),
    add_individuals_meta=IntDistribution(100, 10**8),
    
    prop_species_meta=FloatDistribution(0.0, 1.0),
    speciation_local=FloatDistribution(0.0001, 0.9999),
    speciation_meta=FloatDistribution(0.01, 1.0),
    
    extinction_meta=FloatDistribution(0.01, 1.0),
    env_sigma=FloatDistribution(0.0001, 1000.0),
    trait_sigma=FloatDistribution(0.0001, 1000.0),
    comp_sigma=FloatDistribution(0.0001, 1000.0),
    dispersal_prob=FloatDistribution(0.0, 1.0),
    mutation_rate=FloatDistribution(1e-7, 0.001),
    equilib_escape=FloatDistribution(0.0, 1.0),
    num_basepairs=250,

    alpha=FloatDistribution(1.0, 10**6),
    neut_delta=FloatDistribution(0.0, 1.0),
    env_comp_delta=FloatDistribution(0.0, 1.0),

    init_type='oceanic_island',
    niter=10000,
    niterTimestep=10,
)

In [17]:
sample = prior.sample()
sample

tensor([6.9768e+06, 3.3649e+07, 3.4052e-01, 6.7651e-01, 2.5547e-01, 6.9726e-01,
        4.0881e+01, 8.4624e+01, 2.9749e+02, 8.2208e-01, 4.7713e-04, 5.8151e-01,
        8.3345e+05, 1.2583e-01, 8.5615e-01])

In [18]:
params = prior.get_params(sample)
params

ModelParams(individuals_local=6976828, individuals_meta=40625648, species_meta=13834021, speciation_local=0.6765131950378418, speciation_meta=0.25546738505363464, extinction_meta=0.6972579956054688, env_sigma=40.881072998046875, trait_sigma=84.6239013671875, comp_sigma=297.487548828125, dispersal_prob=0.822076678276062, mutation_rate=0.00047713445383124053, equilib_escape=0.5815138816833496, num_basepairs=250, alpha=833453.1875, neut_delta=0.12582749128341675, env_comp_delta=0.8561491966247559, init_type='oceanic_island', niter=10000, niterTimestep=10)

In [16]:
print(params.model_dump_json(indent=4))

{
    "individuals_local": 34488296,
    "individuals_meta": 110462536,
    "species_meta": 62968478,
    "speciation_local": 0.08231894671916962,
    "speciation_meta": 0.5447497367858887,
    "extinction_meta": 0.41827306151390076,
    "env_sigma": 332.65869140625,
    "trait_sigma": 211.73956298828125,
    "comp_sigma": 327.5611877441406,
    "dispersal_prob": 0.7214080095291138,
    "mutation_rate": 0.0007515561883337796,
    "equilib_escape": 0.32743096351623535,
    "num_basepairs": 250,
    "alpha": 237288.640625,
    "neut_delta": 0.7441015243530273,
    "env_comp_delta": 0.7387944459915161,
    "init_type": "oceanic_island",
    "niter": 10000,
    "niterTimestep": 10
}
